# Game Entities

We define all required game entity types in this notebook.

In [1]:
#| default_exp game.engine.entities

In [2]:
# | hide
%load_ext lab_black

## Imports

In [3]:
# | export
from rl18xx.game.engine.core import *
from rl18xx.game.engine.abilities import Abilities

## Company

In [4]:
# | export


class Company(Entity, Ownable, Passer, Abilities):
    def __init__(
        self, sym=None, name=None, value=None, revenue=0, desc="", abilities=[], **opts
    ):
        Entity.__init__(self)
        Ownable.__init__(self)
        Passer.__init__(self)
        Abilities.__init__(self, abilities)

        self.sym = sym
        self.name = name
        self.value = value
        self.treasury = opts.get("treasury", value)
        self.desc = desc
        self.revenue = revenue
        self.discount = opts.get("discount", 0)
        self.min_auction_price = -self.discount
        self.closed = False
        self.min_price = opts.get(
            "min_price", None if value is None else (value // 2) + (value % 2)
        )
        self.max_price = opts.get("max_price", None if value is None else value * 2)
        self.interval = opts.get("interval", None)  # Array of prices or None
        self.color = opts.get("color", "yellow")
        self.text_color = opts.get("text_color", "black")
        self.type = opts.get("type", None)
        if self.type is not None:
            self.type = self.type
        self.auction_row = opts.get("auction_row", None)

    def __lt__(self, other):
        return (self.min_bid(), self.name) < (other.min_bid(), other.name)

    def id(self):
        return self.sym

    def min_bid(self):
        return self.value - self.discount

    def close(self):
        self.closed = True
        for ability in list(self.all_abilities()):
            self.remove_ability(ability)
        if self.owner:
            if hasattr(self.owner, "companies"):
                self.owner.companies.remove(self)
            self.owner = None

    def is_closed(self):
        return self.closed

    def is_company(self):
        return True

    def is_path(self):
        return False

    # Token handling would need more context about the Token class and related methods
    def find_token_by_type(self, token_type):
        token_ability = next(
            (a for a in self.all_abilities() if a.type == "token"), None
        )
        if token_ability is None:
            raise Exception(f"{self.name} does not have a token")
        if token_ability.from_owner:
            return self.owner.find_token_by_type(token_type)
        return Token(self.owner)  # Assuming a Token class exists

    def __repr__(self):
        return f"<{self.__class__.__name__}: {self.id()}>"

    def get_max_price(self, buyer=None):
        return self.max_price

In [5]:
SV = Company(
    {
        "name": "Schuylkill Valley",
        "sym": "SV",
        "value": 20,
        "revenue": 5,
        "desc": "No special abilities. Blocks G15 while owned by a player.",
        "abilities": [
            {"type": "blocks_hexes", "owner_type": "player", "hexes": ["G15"]}
        ],
        "color": None,
    }
)
SV

<Company: {'name': 'Schuylkill Valley', 'sym': 'SV', 'value': 20, 'revenue': 5, 'desc': 'No special abilities. Blocks G15 while owned by a player.', 'abilities': [{'type': 'blocks_hexes', 'owner_type': 'player', 'hexes': ['G15']}], 'color': None}>

## Bank

In [6]:
# | export


class Bank(Entity, Spender, ShareHolder):
    def __init__(self, cash, log=None, check=True):
        Entity.__init__(self)
        Spender.__init__(self)
        ShareHolder.__init__(self)

        self.cash = cash
        self.log = log or []
        self.broken = False
        self.companies = []
        self.check = check

    def check_cash(self, amount, borrow_from=None):
        if not self.check:
            return
        if self.cash - amount < 0:
            self.break_bank()

    def break_bank(self):
        if not self.broken:
            self.log.append("-- The bank has broken --")
            self.broken = True

    def is_broken(self):
        return self.broken

    @property
    def player(self):
        return None

    @property
    def name(self):
        return "The Bank"

    def __str__(self):
        return f"<{self.__class__.__name__}>"

## Corporation

In [7]:
# | export


class Corporation(
    Abilities, Assignable, Operator, Entity, Ownable, Passer, ShareHolder, Spender
):
    SHARES = [20] + [10] * 8

    def __init__(self, sym=None, name=None, **kwargs):
        Assignable.__init__(self)
        Entity.__init__(self)
        Ownable.__init__(self)
        Passer.__init__(self)
        ShareHolder.__init__(self)
        Spender.__init__(self)

        Abilities.__init__(self, kwargs.get("abilities", []))
        Operator.__init__(self, **kwargs)

        self.name = sym
        self.id = sym
        self.full_name = name

        self.ipo_owner = kwargs.get("ipo_owner", self)
        corp_shares = [
            Share(
                self,
                owner=self.ipo_owner,
                president=(index == 0),
                percent=percent,
                index=index,
            )
            for index, percent in enumerate(kwargs.get("shares", self.SHARES))
        ]
        for share in corp_shares:
            self.ipo_owner.shares_by_corporation.setdefault(self, []).append(share)
        self.share_holders = {}
        self.share_holders[self.ipo_owner] = sum(share.percent for share in corp_shares)

        self.fraction_shares = kwargs.get(
            "fraction_shares",
            any(
                percent not in list(map(lambda x: x.percent, corp_shares[0:2]))
                for percent in corp_shares
            ),
        )
        self.presidents_share = corp_shares[0]
        self.second_share = corp_shares[1] if len(corp_shares) > 1 else None

        self.share_price = None
        self.par_price = None
        self.original_par_price = None
        self.ipoed = False
        self.companies = []
        self.cash = 0
        self.capitalization = kwargs.get("capitalization", "full")
        self.closed = False
        self.float_percent = kwargs.get("float_percent", 60)
        self.float_excludes_market = kwargs.get("float_excludes_market", False)
        self.float_includes_reserved = kwargs.get("float_includes_reserved", False)
        self.floatable = kwargs.get("floatable", True)
        self.floated = False
        self.max_ownership_percent = kwargs.get("max_ownership_percent", 60)
        self.min_price = kwargs.get("min_price")
        self.always_market_price = kwargs.get("always_market_price", False)
        self.needs_token_to_par = kwargs.get("needs_token_to_par", False)
        self.par_via_exchange = None
        self.type = kwargs.get("type", None)
        self.hide_shares = kwargs.get("hide_shares", False)
        self.reservation_color = kwargs.get("reservation_color", None)
        self.price_percent = kwargs.get(
            "price_percent",
            self.second_share.percent
            if self.second_share
            else self.presidents_share.percent / 2,
        )
        self.price_multiplier = (
            self.second_share.percent
            if self.second_share
            else self.presidents_share.percent / 2
        ) / self.price_percent
        self.treasury_as_holding = kwargs.get("treasury_as_holding", False)
        self.corporation_can_ipo = kwargs.get("corporation_can_ipo", None)

    def can_buy(self):
        return True

    def __lt__(self, other):
        self_key = self.sort_order_key()
        other_key = other.sort_order_key()
        if self_key is None:
            return -1
        if other_key is None:
            return 1
        return self_key < other_key

    def sort_order_key(self):
        if self.share_price is None:
            return None
        return [
            -self.share_price.price,
            -self.share_price.coordinates[-1],
            self.share_price.coordinates[0],
            self.share_price.corporations.index(self)
            if self in self.share_price.corporations
            else 0,
            self.name,
        ]

    def counts_for_limit(self):
        return True if self.share_price is None else self.share_price.counts_for_limit()

    def buy_multiple(self):
        return self.share_price.buy_multiple() if self.share_price else False

    def hide_shares(self):
        return self.hide_shares

    def par_price(self):
        if self.closed():
            return None
        return self.share_price if self.always_market_price else self.par_price

    def total_shares(self):
        return 100 / self.share_percent()

    def num_ipo_shares(self):
        return self.ipo_owner.num_shares_of(self)

    def reserved_shares(self):
        return [
            share
            for share in self.ipo_owner.shares_by_corporation[self]
            if not share.buyable
        ]

    def num_ipo_reserved_shares(self):
        return (
            sum(share.percent for share in self.reserved_shares())
            / self.share_percent()
        )

    def num_treasury_shares(self):
        return 0 if self.treasury_as_holding else self.num_shares_of(self)

    def num_player_shares(self):
        return (
            sum(
                holder_value
                for holder, holder_value in self.player_share_holders().items()
            )
            / self.share_percent()
        )

    def num_corporate_shares(self):
        return (
            sum(
                holder_value
                for holder, holder_value in self.corporate_share_holders().items()
            )
            / self.share_percent()
        )

    def num_market_shares(self):
        return (
            sum(
                holder_value
                for holder, holder_value in self.share_holders.items()
                if holder.share_pool()
            )
            / self.share_percent()
        )

    def player_share_holders(self, corporate=False):
        if corporate:
            return {
                holder: holder_value
                for holder, holder_value in self.share_holders().items()
                if holder.player()
                or (
                    corporate
                    and self.corporation_can_ipo
                    and holder.corporation()
                    and holder != self
                )
            }
        return {
            holder: holder_value
            for holder, holder_value in self.share_holders().items()
            if holder.player()
        }

    def ipo_is_treasury(self):
        return self.ipo_owner == self

    def corporate_share_holders(self):
        return {
            holder: holder_value
            for holder, holder_value in self.share_holders().items()
            if holder.corporation() and (holder != self or self.treasury_as_holding)
        }

    def corporate_shares(self):
        return [
            share
            for share in self.shares()
            if share.corporation() == self and not self.treasury_as_holding
        ]

    def ipo_shares(self):
        return [share for share in self.ipo_owner.shares if share.corporation() == self]

    def treasury_shares(self):
        return [
            share
            for share in self.shares()
            if share.corporation() == self and not self.treasury_as_holding
        ]

    def president(self, player):
        if player is None:
            return False
        return self.owner() == player

    def floated(self):
        if not self.floatable:
            return False
        if not hasattr(self, "_floated"):
            self._floated = self.ipo_owner.percent_of(self) <= (
                100
                - self.float_percent
                - (self.percent_in_market() if self.float_excludes_market else 0)
                + (self.percent_in_reserved() if self.float_includes_reserved else 0)
            )
        return self._floated

    def percent_to_float(self):
        if self.floated():
            return 0
        return self.ipo_owner.percent_of(self) - (
            100
            - self.float_percent
            - (self.percent_in_market() if self.float_excludes_market else 0)
            + (self.percent_in_reserved() if self.float_includes_reserved else 0)
        )

    def percent_in_market(self):
        return self.num_market_shares() * self.share_percent()

    def percent_in_reserved(self):
        return self.num_ipo_reserved_shares() * self.share_percent()

    def unfloat(self):
        self._floated = False

    def corporation(self):
        return True

    def receivership(self):
        return self.owner().share_pool() if self.owner() else False

    def __str__(self):
        return f"<{self.__class__.__name__}: {self.id}>"

    def holding_ok(self, share_holder, extra_percent=0):
        common_percent = share_holder.common_percent_of(self) + extra_percent
        return (
            self.share_price and self.share_price.type in ("multiple_buy", "unlimited")
        ) or common_percent <= self.max_ownership_percent

    def all_abilities(self):
        all_abilities = (
            sum((company.all_abilities() for company in self.companies), [])
            + self.abilities()
        )
        if hasattr(self.owner(), "companies"):
            all_abilities += [
                ability
                for company in self.owner().companies()
                for ability in company.all_abilities()
                if "owning_player" in str(ability.when)
            ]
        return all_abilities

    def remove_ability(self, ability):
        if ability.owner() == self:
            super().remove_ability(ability)
        else:
            for company in self.companies:
                company.remove_ability(ability)

    def available_share(self):
        return next(
            (
                share
                for share in self.shares_by_corporation()[self]
                if not share.president()
            ),
            None,
        )

    def presidents_percent(self):
        return self.presidents_share.percent

    def share_percent(self):
        return (
            self.forced_share_percent
            if hasattr(self, "forced_share_percent")
            else (
                self.second_share.percent
                if self.second_share
                else self.presidents_share.percent / 2
            )
        )

    def player(self):
        chain = {self.owner(): True}
        current = self.owner()
        while current and current.corporation():
            if not current.owner():
                return None
            current = current.owner()
            if current in chain:
                return None
            chain[current] = True
        return current.player() if current and current.player() else None

    def closed(self):
        return self.closed

    def close(self):
        if self.share_price:
            self.share_price.corporations().remove(self)
        self.closed = True
        self.ipoed = False
        self.floated = False
        self.owner = None

    def reopen(self):
        self.closed = False

In [8]:
# Example usage

c = Corporation(
    {
        "float_percent": 60,
        "sym": "PRR",
        "name": "Pennsylvania Railroad",
        "logo": "18_chesapeake/PRR",
        "simple_logo": "1830/PRR.alt",
        "tokens": [0, 40, 100, 100],
        "coordinates": "H12",
        "color": "#32763f",
    }
)
c.shares_by_corporation

{<__main__.Corporation at 0x7f1766f13a50>: [<rl18xx.game.engine.core.Share at 0x7f1766d66210>,
  <rl18xx.game.engine.core.Share at 0x7f1766c2e610>]}

## Depot

In [9]:
# | export
class Depot(Entity):
    def __init__(self, trains, game):
        super().__init__()

        self.game = game
        self.trains = trains
        for train in self.trains:
            train.owner = self
        self.upcoming = self.trains.copy()
        self.discarded = []
        self.bank = self.game.bank
        self.depot_trains_cache = None

    def export(self):
        train = self.upcoming[0]
        self.game.log.append(f"-- Event: A {train.name} train exports --")
        self.game.remove_train(train)
        self.game.phase.buying_train(None, train, self)

    def export_all(self, name, silent=False):
        if not silent:
            self.game.log.append(f"-- Event: All {name} trains are exported --")
        while self.upcoming and self.upcoming[0].name == name:
            train = self.upcoming[0]
            self.game.remove_train(train)
            self.game.phase.buying_train(None, train, self)

    def reclaim_all(self, name):
        self.game.log.append(
            f"-- Event: All {name} trains are discarded to the Bank Pool --"
        )
        while self.upcoming and self.upcoming[0].name == name:
            train = self.upcoming[0]
            self.reclaim_train(train)
            self.game.phase.buying_train(None, train, self)

    def reclaim_train(self, train):
        if train.owner is None:
            return
        self.game.remove_train(train)
        train.owner = self
        if self.game.discarded_train_placement == "discard" and not train.obsolete:
            self.discarded.append(train)
        self.depot_trains_cache = None

    def min_price(self, corporation, ability=None):
        return min(
            train.min_price(ability=ability) for train in self.available(corporation)
        )

    def min_depot_train(self):
        return min(self.depot_trains(), key=lambda train: train.price)

    def min_depot_price(self):
        train = self.min_depot_train()
        return (
            min(variant["price"] for variant in train.variants.values()) if train else 0
        )

    def max_depot_price(self):
        train = max(self.depot_trains(), key=lambda train: train.price, default=None)
        return (
            max(variant["price"] for variant in train.variants.values()) if train else 0
        )

    def unshift_train(self, train):
        train.owner = self
        self.upcoming.insert(0, train)
        self.depot_trains_cache = None

    def remove_train(self, train):
        self.upcoming.remove(train)
        self.discarded.remove(train)
        self.depot_trains_cache = None

    def forget_train(self, train):
        self.trains.remove(train)
        self.upcoming.remove(train)
        self.discarded.remove(train)
        self.depot_trains_cache = None

    def add_train(self, train):
        train.owner = self
        self.trains.append(train)
        self.upcoming.append(train)
        self.depot_trains_cache = None

    def insert_train(self, train, index=0):
        train.owner = self
        self.trains.append(train)
        self.upcoming.insert(index, train)
        self.depot_trains_cache = None

    def depot_trains(self, clear=False):
        if clear or self.depot_trains_cache is None:
            self.depot_trains_cache = [self.upcoming[0]] + [
                t for t in self.upcoming if self.game.phase.available(t.available_on)
            ]
            self.depot_trains_cache = list(
                dict.fromkeys(self.depot_trains_cache + self.discarded).keys()
            )
        return self.depot_trains_cache

    def available(self, corporation):
        return self.depot_trains() + self.other_trains(corporation)

    def other_trains(self, corporation):
        all_others = [
            train
            for train in self.trains
            if train.buyable and train.owner not in [corporation, self, None]
        ]
        if not self.game.ALLOW_TRAIN_BUY_FROM_OTHER_PLAYERS:
            all_others = [
                train for train in all_others if train.owner.owner == corporation.owner
            ]
        return all_others

    @property
    def cash(self):
        return self.bank.cash

    @cash.setter
    def cash(self, new_cash):
        self.bank.cash = new_cash

    @property
    def name(self):
        return "The Depot"

    def empty(self):
        return not self.depot_trains()

    def player(self):
        return None

## Minor

In [10]:
# | export


class Minor(Abilities, Assignable, Operator, Entity, Ownable, Passer, Spender):
    def __init__(self, sym, name, abilities=None, **kwargs):
        Assignable.__init__(self)
        Entity.__init__(self)
        Ownable.__init__(self)
        Passer.__init__(self)
        Spender.__init__(self)

        Abilities.__init__(self, kwargs.get("abilities", []))
        Operator.__init__(self, **kwargs)

        self.name = sym
        self.full_name = name
        self.floated = False
        self.closed = False
        self.type = kwargs.get("type")
        self.reservation_color = kwargs.get("reservation_color")

    def companies(self):
        return []

    def id(self):
        return self.name

    def minor(self):
        return True

    def total_shares(self):
        return 1

    def floated(self):
        return self.floated

    def float(self):
        self.floated = True

    def inspect(self):
        return f"<{self.__class__.__name__}: {self.id()}>"

    def closed(self):
        return self.closed

    def share_price(self):
        pass

    def par_price(self):
        pass

    def num_shares_of(self, corporation, ceil=True):
        return 0

    def share_percent(self):
        return 100

    def president(self, player):
        if player:
            return self.owner() == player
        return False

    def close(self):
        self.closed = True
        self.floated = False
        self.owner = None

    def reopen(self):
        self.closed = False

## PlayerInfo

In [14]:
# | export
class PlayerInfo:
    def __init__(self, round_name, turn, round_no, player_value):
        self.round_name = round_name
        self.turn = turn
        self.round_no = round_no
        self.value = player_value

    def round(self):
        if self.round_name in ["AR", "MR", "OR", "DEV", "BUST"]:
            return f"{self.round_name} {self.turn}.{self.round_no}"
        else:
            return f"{self.round_name} {self.turn}"

## Player

In [15]:
# | export
class Player(Entity, Passer, ShareHolder, Spender):
    def __init__(self, id, name):
        super().__init__()
        self.id = id
        self.name = name
        self.cash = 0
        self.companies = []
        self.history = []
        self.unsold_companies = []

    def value(self):
        return (
            self.cash
            + sum(s.price for s in self.shares if s.corporation.ipoed)
            + sum(c.value for c in self.companies)
        )

    def owner(self):
        return self

    def player(self):
        return self

    def corporation(self):
        return None

    def __eq__(self, other):
        return isinstance(other, Player) and self.name == other.name

    def player(self):
        return True

    def __str__(self):
        return f"{self.__class__.__name__} - {self.name}"

    def __repr__(self):
        return f"<{self.__class__.__name__} - {self.name}>"

# Testing